# Loading Geospatial Data with GeoPandas: Supported Data Types

The data from this notebook comes from the [NYC open data portal](https://opendata.cityofnewyork.us/).

In [ ]:
# Import necessary libraries
import geopandas as gpd
import pandas as pd

# Many file types, same read function: ```gpd.read_file()```

## Shape files

In [ ]:
# Designated pedestrian zones in NYC
shapefile = "../../geopandas_101_DATA/Forrest/data/Pedestrian Zone Shapefile (Tabular)_20241220/geo_export_a7bb075a-41dc-445f-8244-8430e90a8dde.shp"
shapefile_gdf = gpd.read_file(shapefile)

In [ ]:
shapefile_gdf

In [ ]:
shapefile_gdf.explore()

## GeoJSON

In [ ]:
# NYC unofficial neighborhoods
geojson_fp = "https://raw.githubusercontent.com/HodgesWardElliott/custom-nyc-neighborhoods/refs/heads/master/custom-pedia-cities-nyc-Mar2018.geojson"
geojson_gdf = gpd.read_file(geojson_fp)

In [ ]:
geojson_gdf.explore()

## Geo Packages

In [ ]:
# World rivers
# gpkg_fp = "https://ngageoint.github.io/GeoPackage/examples/rivers.gpkg"
gpkg_fp = "../../geopandas_101_DATA/Soma/data/rivers.gpkg"
gpkg_gdf = gpd.read_file(gpkg_fp)
gpkg_gdf.explore()

## Flat Geo Files

In [ ]:
# US counties
# fgb_fp = "https://github.com/flatgeobuf/flatgeobuf/raw/refs/heads/master/test/data/UScounties.fgb"
fgb_fp = "../../geopandas_101_DATA/Soma/data/UScounties.fgb"
fgb_gdf = gpd.read_file(fgb_fp)
fgb_gdf.explore()

# Using SQL tools

Even though you can do everything in Python using GeoPandas, there are some really good reasons to combine it with the use of SQL or a tool like DuckDB instead:


Efficiency

* **DuckDB can be faster** for certain operations — particularly **spatial joins and aggregations**.
* SQL queries are executed within a **high-performance, in-memory engine** (DuckDB), which can outperform pandas/GeoPandas when handling large datasets.


Simplified data prep

* SQL can **condense data cleaning and transformation** (e.g., joins, filters, type casting) into **fewer lines** of declarative logic.
* For example, filtering by date and doing a spatial intersection can be written as one SQL query instead of multiple pandas/GeoPandas steps.


Avoiding memory overload

* With GeoPandas, you must **load entire datasets into memory**.
* DuckDB allows you to **process data on demand** — which can be more scalable, especially with large files (like zipped CSVs or remote Parquet files).


Combining diverse file types

* SQL (via DuckDB) can read **remote or local files**, such as Parquet, CSV, or even GeoPackages, and **query them together**.
* This is harder to do cleanly in pure Python.

**In short**: It's not about replacing GeoPandas — it’s about giving you more tools for the job; extending the ecosystem. You might use DuckDB/SQL to gain speed, memory efficiency, and cleaner preprocessing — especially for **larger datasets** or **more complex queries**.


## Post GIS

In [ ]:
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine
import duckdb


# Define PostGIS connection parameters
host = "your_host"           # e.g., "localhost" or your database IP
database = "your_database"   # Database name
user = "your_user"           # Username
password = "your_password"   # Password
port = "5432"                # Default PostgreSQL port

# SQLAlchemy connection string for PostGIS
postgis_connection = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Create SQLAlchemy engine
engine = create_engine(postgis_connection)

# Define SQL query to read spatial data
# Replace 'your_table_name' with the actual table containing spatial data
query = "SELECT * FROM your_table_name"

# Load the PostGIS table into a GeoDataFrame
postgis_gdf = gpd.read_postgis(query, con=engine, geom_col="geometry", crs="EPSG:4326")

postgis_gdf.head()

## DuckDB

In [ ]:
# !pipenv install duckdb

In [ ]:
import duckdb

# Initialize a DuckDB connection
con = duckdb.connect()
con.query('INSTALL spatial')
con.query('LOAD spatial')

### Querying remote CSVs

The code below queries a series of six CSV files that are containd in a zip file, which is hosted in AWS. Those CSVs contain all the bike rides of the NYC Citi Bike system for the for the month of June 2024. Each CSV has the following columns:

*ride_id, rideable_type, started_at, ended_at, start_station_name, start_station_id, end_station_name, end_station_id, start_lat, start_lng, end_lat, end_lng, member_casual*

Instead of downloading the whole zip file, unzipping it, etc., we can query the CSVs directly and only download the data we need.

In [ ]:
# Define a DuckDB SQL query
duckdb_query = """
SELECT *, ST_AsText(ST_Point(column09, column08)) as geometry
FROM read_csv('https://s3.amazonaws.com/tripdata/202406-citibike-tripdata.zip', ignore_errors=true)
"""

In [ ]:
# Execute DuckDB query and convert to a Pandas DataFrame
duckdb_df = con.query(duckdb_query).to_df()

In [ ]:
print(duckdb_df.shape)
duckdb_df.head(5)

In [ ]:
duckdb_gdf = gpd.GeoDataFrame(
    duckdb_df,
    geometry=gpd.GeoSeries.from_wkt(duckdb_df["geometry"]),
    crs="EPSG:4326"
)


In [ ]:
duckdb_gdf.columns = [
'ride_id', 'rideable_type', 'started_at', 'ended_at', 
    'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 
    'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual',
    'geometry'
]

In [ ]:
duckdb_gdf.info()

In [ ]:
duckdb_gdf.head(1000).explore()

### Querying remote GeoJSON

In [ ]:
server = "https://raw.githubusercontent.com/HodgesWardElliott/custom-nyc-neighborhoods/refs/heads/master/custom-pedia-cities-nyc-Mar2018.geojson"

# Instead of calling the whole dataset using gdp.read_file(), like this:
# neighborhoods = gpd.read_file()

# we instead run a limited query:xqd
nbh_query = f"""
SELECT *
FROM ST_Read('{server}')
limit 10
"""
print(nbh_query)

In [ ]:
duckdb_df = con.query(nbh_query).to_df()
duckdb_df

Now, let's run a simultaneous online query of both the CSVs and the GeoJSON files above.

This SQL query is designed to perform a spatial analysis that links bicycle trip data to New York City neighborhoods. It illustrates how to integrate geospatial data with real-world activity data using DuckDB, a high-performance analytical database, and ST\_\* spatial functions.

The query begins by selecting two data sources: a GeoJSON file containing custom-defined NYC neighborhoods (each with a name and polygon geometry), and a zipped CSV file of Citi Bike trips for June 2024. It then performs a spatial join between these two datasets using the `ST_Intersects` function. This spatial predicate checks whether a bike trip’s starting point (reconstructed from columns 9 and 8 in the trip data—likely longitude and latitude, respectively) falls within the geometry of any NYC neighborhood.

From there, it filters the results to keep only those trips that occurred on June 15th, 2024. The query groups the results by neighborhood name and geometry, and counts the number of trips that fall into each neighborhood on that day.

Finally, the result is a table where each row corresponds to a neighborhood, and includes:

* the number of Citi Bike trips that started within its boundaries on June 15, 2024,
* the neighborhood name,
* and its geometry in well-known text (WKT) format, ready for mapping or further analysis.

This analysis is useful for understanding spatial patterns in bike usage—such as identifying which neighborhoods had the most activity on a given day. It demonstrates how spatial joins, data filtering, and aggregation can combine to produce valuable geographic insights.


In [ ]:
nbh_query = """
SELECT 
    count(b.column00) as count,           -- Count the number of Citi Bike trips that matched the spatial condition
    n.neighborhood,                       -- Select the neighborhood name from the neighborhoods GeoJSON
    ST_AsText(n.geom) as geom            -- Convert the neighborhood geometry to Well-Known Text (WKT) format for easy readability/output
FROM 
    ST_Read(
        'https://raw.githubusercontent.com/HodgesWardElliott/custom-nyc-neighborhoods/refs/heads/master/custom-pedia-cities-nyc-Mar2018.geojson'
    ) n                                  -- Load the NYC neighborhood geometries from a remote GeoJSON file; alias this table as 'n'
JOIN 
    read_csv(
        'https://s3.amazonaws.com/tripdata/202406-citibike-tripdata.zip', 
        ignore_errors=true
    ) b                                  -- Load the zipped Citi Bike trip data CSV from AWS S3; alias this table as 'b'
ON 
    ST_Intersects(n.geom, ST_Point(column09, column08)) 
                                         -- Perform a spatial join: check if the starting point of a bike trip 
                                         -- (constructed from column09 = longitude, column08 = latitude) intersects the neighborhood geometry
WHERE 
    CAST(column02 AS DATE) = DATE '2024-06-15'
                                         -- Filter to include only bike trips that took place on June 15, 2024
GROUP BY 
    n.neighborhood, n.geom               -- Group the results by neighborhood name and geometry
""" 


In [ ]:
nbh_df = con.query(nbh_query).to_df()
nbh_df

In [ ]:
join_gdf = gpd.GeoDataFrame(
    nbh_df,
    geometry=gpd.GeoSeries.from_wkt(nbh_df["geom"]),
    crs="EPSG:4326"
)

In [ ]:
join_gdf

In [ ]:
join_gdf.explore()

# Parquet files with ```gpd.read_parquet()```

In [ ]:
gdf = gpd.read_parquet('../../geopandas_101_DATA/Forrest/data/es_cn.parquet')
gdf.info()

In [ ]:
gdf.head(100).explore()

Somebody he mentioned took the City Bike zip data and transformed int a parquet file [here](https://source.coop/repositories/zluo43/citibike/description). He suggested it as a good tool to practice how to use the ```gdp.read_parquet()``` function. I think his point is that the files themselves are already the results of queries. Each parquet file file in there is the result of a query, so by downoading them it is equivalent to having alrady run the queries.